# Este código limpia y acota las bases de datos con los negocios pertenecientes al código 72 del NAICS

Aquí quitaré las columnas que no necesito y mantendré solamente los datos del Distrito Federal / Ciudad de México

## Importación de librerias y carga de las bases de datos

In [1]:
import pandas as pd
import os

In [2]:
#Definimos la carpeta en donde están los datos
carpeta = r"C:\Users\santo\Documents\CIDE\Tesis\datos_turismo"

In [3]:
#Como los nombres de las columnas no son iguales, hago un diccionario que mapea cualquier nombre posible al nombre estándar que ocuparé
cols_map = {
    # Mapeos para "id"
    'id': 'id',
    'ID': 'id',
    
    # Mapeos para "latitud"
    'latitud': 'latitud',
    'Latitud': 'latitud',
    
    # Mapeos para "longitud"
    'longitud': 'longitud',
    'Longitud': 'longitud',
    
    # Mapeos para "cve_ent"
    'cve_ent': 'cve_ent',
    'Clave entidad': 'cve_ent'
}
posible_cols = list(cols_map.keys())

In [9]:
#Listar los archivos que ocuparé
archivos = ["denue_inegi_72_2015.csv", "denue_inegi_72_2019_04.csv"]

#Lista vacia para guardar los dataframes
dataframe = []

#Leer y guardar las bases
for archivo in archivos:
    ruta_completa = os.path.join(carpeta, archivo)
    # Extraer solo el año (los primeros 4 dígitos después del último "_")
    año_str = archivo.split("_")[-2] if archivo.endswith(".csv") and archivo.split("_")[-1].isdigit() else archivo.split("_")[-1].replace(".csv", "")
    fecha_recabada = pd.to_datetime(año_str, format="%Y").to_period("Y")
    df = (pd.read_csv(ruta_completa, usecols = lambda col: col in posible_cols, encoding="latin-1")#Con a función de lambda estoy creando un argumento booleano que recibe una columna del archivo y regresa true si esa columna está en la lista "posible_cols". Se  le aplica esa función a cada nombre de columna del archivo, y solo carga las columnas para las que la función devuelve True.
          .rename(columns = cols_map) #Renombramos las columnas con el diccionario
          .query("cve_ent == 9") #Quedarme con los datos solamente de la Ciudad de México
          .assign(year = fecha_recabada)
    ) 
    dataframe.append(df)

#Concatenar los archivos / Unir las bases de datos en una sola
df_concatenado = pd.concat(dataframe, ignore_index=True)
df_concatenado

,id,cve_ent,latitud,longitud,year
0,704479,9,19.401299,-99.135284,2015
1,741039,9,19.468438,-99.159188,2015
2,747931,9,19.495539,-99.118570,2015
3,684252,9,19.469465,-99.159156,2015
4,749788,9,19.412505,-99.090889,2015
...,...,...,...,...,...
112230,8584276,9,19.307901,-99.163911,2019
112231,7743173,9,19.449317,-99.162096,2019
112232,736223,9,19.328171,-99.043536,2019
112233,8368771,9,19.396080,-99.171465,2019


## Preparamos la base para exportarlo

In [10]:
df_final = df_concatenado.drop(columns=['cve_ent'])

carpeta_final = r"C:\Users\santo\Documents\CIDE\Tesis\datos_unidos"

df_final.to_csv(os.path.join(carpeta_final, "turismo_limpio.csv"), index = False, encoding='utf-8-sig')